# Image discriminator

This notebook contains code to run an attribute classifier for the 40 CelebA attributes on a dataset of images, in preparation of the SVM of InterFaceGAN.

Given a proportion `p` (percentage), the notebook runs the classifier on the dataset, and for each of the 40 attributes, it saves the scores of all the images in a JSON file, then labels the images with the top `p`% scores with +1 and the images with the lowest `p`% scores with -1, and finally saves the resulting labels in a JSON file.

The classifier was provided by Gwilherm Lesné (pre-trained, based on EfficientNet).

The list of attributes is as follows:

<details>
  <summary>Click to expand</summary>
  
  | Attribute # | Name |
  |-------------|------|
  |0|`5_o_Clock_Shadow`|
  |1|`Arched_Eyebrows`|
  |2|`Attractive`|
  |3|`Bags_Under_Eyes`|
  |4|`Bald`|
  |5|`Bangs`|
  |6|`Big_Lips`|
  |7|`Big_Nose`|
  |8|`Black_Hair`|
  |9|`Blond_Hair`|
  |10|`Blurry`|
  |11|`Brown_Hair`|
  |12|`Bushy_Eyebrows`|
  |13|`Chubby`|
  |14|`Double_Chin`|
  |15|`Eyeglasses`|
  |16|`Goatee`|
  |17|`Gray_Hair`|
  |18|`Heavy_Makeup`|
  |19|`High_Cheekbones`|
  |20|`Male`|
  |21|`Mouth_Slightly_Open`|
  |22|`Mustache`|
  |23|`Narrow_Eyes`|
  |24|`No_Beard`|
  |25|`Oval_Face`|
  |26|`Pale_Skin`|
  |27|`Pointy_Nose`|
  |28|`Receding_Hairline`|
  |29|`Rosy_Cheeks`|
  |30|`Sideburns`|
  |31|`Smiling`|
  |32|`Straight_Hair`|
  |33|`Wavy_Hair`|
  |34|`Wearing_Earrings`|
  |35|`Wearing_Hat`|
  |36|`Wearing_Lipstick`|
  |37|`Wearing_Necklace`|
  |38|`Wearing_Necktie`|
  |39|`Young`|
</details>

In [1]:
import os

**Settings**

In [2]:
if not os.path.exists('test_dataset'):
    os.makedirs('test_dataset')

if not os.path.exists('test_dataset/Labels'):
    os.makedirs('test_dataset/Labels')

In [3]:
proportion = 2 # Percentage of images to keep for each class


dataset_path = 'test_UnlabeledImages' # Path to dataset after extraction from the archive
output_path ='test_dataset/Labels' # Path where the output JSON files will be placed

Imports

In [4]:
from torchvision import models, transforms
import torch
import torch.nn as nn
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
from os.path import isfile
from tqdm import tqdm
import json
import argparse

## Downloading and loading the discriminator

In [5]:
device_name = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_name)
print(device)

cuda


In [6]:
class EfficientNetB0(nn.Module):
    def __init__(self):
        super(EfficientNetB0, self).__init__()
        self.model = models.efficientnet_b0(pretrained=True)
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(1,1))
        self.head = nn.Sequential(
            nn.Linear(1280, 1024),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.2),
            nn.Linear(1024 , 256),
            nn.BatchNorm1d(256),
            nn.Dropout(0.2),
            nn.Linear(256 , 40)
            )

    def forward(self,x):
        x = self.model.features(x)
        #-----#
        x = self.avgpool(x)
        x = x.view(x.shape[0], -1)
        x = self.head(x)
        return x


Downloading the weights of the discriminator:

In [7]:
# Loading Discriminator

net = EfficientNetB0()
net.load_state_dict(torch.load('atclas2.pt'))
net = net.to(device)
net.eval()

C:\Users\nmadali\AppData\Local\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\nmadali\AppData\Local\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


EfficientNetB0(
  (model): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (scale_

## Classification

In [8]:
def run_classification(dataset_path, output_path, proportion, batch_size=10):
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    if not os.path.isdir(output_path):
        raise ValueError("Output path must be a directory, not overwriting existing file")
    filenames = [f for f in os.listdir(dataset_path) if isfile(os.path.join(dataset_path, f))]
    filenames_and_scores = [[] for _ in range(40)]
    totensor = transforms.ToTensor()

    for i in tqdm(range(0, len(filenames), batch_size)):
        # Load the batch
        batch = torch.zeros((batch_size, 3, 1024, 1024))
        batch_filenames = filenames[i:min(i+batch_size, len(filenames))]
        for j, f in enumerate(batch_filenames):
            path = os.path.join(dataset_path, f)
            if not isfile(path):
                continue
            img = Image.open(path)
            batch[j] = 255*totensor(img)
            img.close()
        batch = batch.to(device)

        # Feed the batch to the network and store the results
        with torch.inference_mode():
            out = net(batch).cpu()
        for j, f in enumerate(batch_filenames):
            for att in range(40):
                filenames_and_scores[att].append((f, out[j, att].item()))

    # Store the results in JSON files
    for att in range(40):
        with open(os.path.join(output_path, f"att{att}_scores.json"), "w") as outfile:
            json.dump(dict(filenames_and_scores[att]), outfile)

        # Create two classes with the samples classified with the highest confidence
        filenames_and_scores[att].sort(key=lambda p: p[1])
        num_top = int(proportion*len(filenames)/100)
        filenames_minus1 = [(p[0], -1) for p in filenames_and_scores[att][:num_top]]
        filenames_plus1 = [(p[0], 1) for p in filenames_and_scores[att][-num_top:]]
        with open(os.path.join(output_path, f"att{att}_labels.json"), "w") as outfile:
            json.dump(dict(filenames_minus1 + filenames_plus1), outfile)

In [9]:
run_classification(dataset_path, output_path, proportion)

100%|██████████████████████████████████████████████████████████████████████████████| 5000/5000 [17:26<00:00,  4.78it/s]
